In [1]:
# Dependencies for the MCP server
%pip install impyla python-dotenv "mcp[cli]" --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies for the application
%pip install openai-agents --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Disable tracing
from agents import set_tracing_disabled

set_tracing_disabled(True)

In [4]:
import asyncio

from openai import AsyncOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel
from agents.mcp import MCPServer, MCPServerStdio

In [5]:
# Configuring AI Inference service hosted model Endpoint
import json

API_KEY = json.load(open("/tmp/jwt"))["access_token"]
MODEL_NAME = "meta/llama-3.1-8b-instruct"  # Update this for your Cloudera AI Inference service Endpoint
BASE_URL = "https://ml-2dad9e26-62f.cloudera.site/namespaces/serving-default/endpoints/llama3-8b-throughput/v1/"  # Update this for your Cloudera AI Inference service Endpoint

client = AsyncOpenAI(api_key=API_KEY, base_url=BASE_URL)
model = OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client)

In [6]:
async def run(mcp_server: MCPServer):
    agent = Agent(
        name="Assistant",
        instructions="Answer questions, use tools that are available for you.",
        model=model,
        mcp_servers=[mcp_server],
    )

    message = "What information can I learn form the `airlines_snappy` table? I'm looking for a quick answer, don't ask for clarifying questions."
    print(f"Running: {message}")
    result = await Runner.run(starting_agent=agent, input=message)
    print(f"\nAnswer: {result.final_output}")

In [7]:
# Configuring MCP Server and Impala endpoint
async def main():
    async with MCPServerStdio(
        name="Cloudera Iceberg MCP Server",
        params={
            "command": "python",
            "args": ["/home/cdsw/iceberg-mcp-server/server.py"],
            "env": {
                "IMPALA_HOST": "coordinator-default-impala.example.cloudera.site",  # Update this for your Impala host
                "IMPALA_USER": "username",
                "IMPALA_PASSWORD": "password",
            },
        },
    ) as server:
        await run(server)

In [8]:
await main()

Running: What information can I learn form the `airlines_snappy` table? I'm looking for a quick answer, don't ask for clarifying questions.

Answer: The `airlines_snappy` table contains information about flights, including their departure and arrival times, delays, and various other details. It includes 31 columns, including carrier and flight number, departure and arrival times, airline delays, and other factors that could impact the flight.
